<a href="https://colab.research.google.com/github/nechebarrena/HOML/blob/main/HOML_cap_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CAPITULO 14 - HANDS ON MACHINE LEARNING**
## **Vision por computadora usando CNN's**

A lo largo de este capitulo el libro se propone analizar las redes neuronales convolucionales (CNN) y su implementacion con TensorFlow y Keras aplicadas a la vision por computadora. En capitulos posteriores va a utilizar tambien CNN's para otro tipo de tareas.

### **Capas convolucionales**
Las capas convolucionales son los ladrillos elementales para construir una CNN. 
Las neuronaes en estas capas no estan conectadas a todos los datos de entrada (que de ahora en mas vamos a suponer que es una matriz donde cada elemento es un pixel y por lo tanto representa una imagen) sino a un pequeño grupo llamado _receptive fields_. Esta forma de conectar las neuronas permite que la red se concentre en atributos de pequeña escala.

_ver lo de zero padding_

En general los campos de vision o _receptive fields_ entre dos neuronas contiguas se suelen solapar, tal como se muestra en la figura $14.3$ del libro, sin embargo el tamaño del solapamiento puede ser elegido segun la arquitectura. El parametro de control que nos permite seleccionar cuan solapado esta el campo de vision de dos neuronas contiguas se denomina _stride_. Aumentar el stride reduce el solapamiento y esto, obviamente, reduce la cantidad de neuronas en la capa. Esto se ejemplifica en la figura $14.4$ del libro.

